In [ ]:
from qdtuner import SingleQuantumDotTuner

In [ ]:
QDTuner = SingleQuantumDotTuner(
    device_config='../device_config.yml',
    station_config='../station_config.yml',
    tuner_config='../tuner_config.yml',
    qcodes_config='../qcodes_config.yml'
)

In [ ]:
QDTuner.bias_device(Vbias=0.005) # 0.5 mV

In [ ]:
# Set maxV equal to None if you want to sweep 
# up to the max voltage you set in device config
# It'll stop if it hits max current or max gate voltage (it should)
QDTuner.check_turn_on(minV=1, maxV=2.5, dV=0.05, delay=0.2)

In [ ]:
QDTuner.device_info

In [ ]:
import time
QDTuner.station.sim900.write('*DCL')  # device clear
QDTuner.station.sim900.write('FLSH')  # flush port buffers
QDTuner.station.sim900.write('SRST')  # SIM reset (causes 100 ms delay)
time.sleep(0.5)

In [ ]:
# Set maxV equal to None if you want to use
# saturation voltage found in turn on phase.
QDTuner.check_pinch_offs(minV=None, maxV=None, dV=0.01, delay=0.5)

In [ ]:
QDTuner.device_info

In [ ]:
import time
QDTuner.station.sim900.write('*DCL')  # device clear
QDTuner.station.sim900.write('FLSH')  # flush port buffers
QDTuner.station.sim900.write('SRST')  # SIM reset (causes 100 ms delay)
time.sleep(0.5)

In [ ]:
# B1 is outer, B2 is inner loop
QDTuner.barrier_barrier_sweep(B1='LB', B2='RB', voltage_configuration={}, dV=0.002, delay=0.5)

In [ ]:
QDTuner.coulomb_blockade(P='P', voltage_configuration={'LB': 1.35, 'RB': 1.29}, dV=0.1)

In [ ]:
QDTuner.coulomb_diamonds(ohmic='S', gate='STL', dV_gate=0.5, dV_ohmic=0.002)